# Imports

In [10]:
import api
import filter_data
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [11]:
#Set endpoint/URL and fetch data
url = 'https://api.jordbruksverket.se/rest/povapi/graderingar?fran=2010-08-04&till=2019-11-22&groda=Höstvete&skadegorare=blad'

#Takes a few seconds to process
df = api.fetch_data(url)

In [12]:

#Not needed as we are collecting for only this right now, altough could be useful in the future if we decide to fetch all of the data at once
#Can create a filter.py module similar to api.py

df = api.sweref99tm_to_wgs84(df)


In [ ]:
df_flat = filter_data.flatten_data(df)
df_blekinge = filter_data.filter_area(df_flat, 'Blekinge')
print(df_flat.head(20))

In [ ]:
def plot_pest_time_series_group(record: pd.Series) -> None:
    timestamps = record["timestamps"]
    values = record["measurement_values"]

    sorted_indices = np.argsort(timestamps)
    sorted_timestamps = np.array(timestamps)[sorted_indices]
    sorted_values = np.array(values)[sorted_indices]

    if len(sorted_timestamps) > 0 and pd.notnull(sorted_timestamps[0]):
        year_str = sorted_timestamps[0].year
    else:
        year_str = "N/A"

    title_str = (
        f"Delomrade: {record['area']} | Year: {year_str} | "
        f"Crop: {record['crop']} | Pest: {record['pest']} | "
        f"Coordinates: {record['latitud']}, {record['longitud']}"
    )

    # Create the plot
    plt.figure(figsize=(10, 6))
    plt.plot(sorted_timestamps, sorted_values, marker='o', linestyle='-')
    plt.xlabel("Date")
    plt.ylabel("Pest Level")
    plt.title(title_str)
    plt.grid(True)
    plt.tight_layout()
    plt.show()


bladflacksvampar = filter_data.filter_by_pest(df_blekinge, 'Bladfläcksvampar')

print(len(bladflacksvampar))

for i in range(30):
    record = bladflacksvampar.iloc[i]
    plot_pest_time_series_group(record)